In [ ]:
import pymongo
import os
import json
from dotenv import load_dotenv

load_dotenv()

In [ ]:
_URI = f"mongodb://{os.environ['MONGODB_USERNAME']}:{os.environ['MONGODB_PASSWORD']}@{os.environ['MONGODB_HOST']}:{os.environ['MONGODB_PORT']}/{os.environ['MONGODB_DATABASE']}"

In [ ]:
client = pymongo.MongoClient(_URI)
client.list_database_names()

In [ ]:
db = client["function_calling"]
collection = db["gorilla_openfunctions"]

In [ ]:
import ast
def recursively_dump_to_json(input_json_str):
    # Parse the outer JSON string
    input_json_str = str(input_json_str)
    input_json_str = input_json_str.strip()
    
    try:
        outer_data = json.loads(input_json_str)
    except json.JSONDecodeError as e:
        print(f"Error parsing JSON: {e}")
        print(f"Input JSON: {input_json_str}")
        return {
            "error": str(e),
            "data": input_json_str
        }

    # Function to recursively convert inner JSON strings
    def convert_inner_json(obj):
        if isinstance(obj, str):
            obj = obj.strip()
            if (obj.startswith("{") and obj.endswith("}")) or (obj.startswith("[") and obj.endswith("]")):
                try:
                    return ast.literal_eval(obj)
                except Exception as e:
                    print(f"Error parsing: {e}")
                    print(f"Input string: {obj}")
                    raise e
            
            return obj
            
        elif isinstance(obj, list):
            return [convert_inner_json(item) for item in obj]
        
        elif isinstance(obj, dict):
            return {key: convert_inner_json(value) for key, value in obj.items()}

    # Recursively convert inner JSON strings
    converted_data = convert_inner_json(outer_data)
    
    return dict(converted_data)

In [ ]:
with open(os.environ["TEST_FILE_PATH"]) as f:
    for num, data in enumerate(f.readlines()):
        
        if data.strip() == "":
            continue

        data = recursively_dump_to_json(data)
        
        try:
            collection.insert_one(
                {
                    "split": "test",
                    "line": num+1,
                    "data": data
                }
            )
        except pymongo.errors.DuplicateKeyError:
            print(f"Duplicate key error at line {num+1}")
            # Get the duplcate element
            duplicate_element = collection.find_one({"split": "test", "line": num+1})
            if duplicate_element["data"] != data:
                print("Duplicate element is different")
                print(f"Old element: {duplicate_element}")
                print(f"New element: {data}")
                print("Deleting old element and inserting new element")
                collection.delete_one({"split": "test", "line": num+1})
                collection.insert_one(
                    {
                        "split": "test",
                        "line": num+1,
                        "data": data
                    }
                )
            else:
                print("Duplicate element is same, skipping")

In [ ]:
with open(os.environ["TRAIN_FILE_PATH"]) as f:
    for num, data in enumerate(f.readlines()):
        
        if data.strip() == "":
            continue

        data = recursively_dump_to_json(data)
        
        try:
            collection.insert_one(
                {
                    "split": "train",
                    "line": num+1,
                    "data": data
                }
            )
        except pymongo.errors.DuplicateKeyError:
            print(f"Duplicate key error at line {num+1}")
            # Get the duplcate element
            duplicate_element = collection.find_one({"split": "test", "line": num+1})
            if duplicate_element["data"] != data:
                print("Duplicate element is different")
                print(f"Old element: {duplicate_element}")
                print(f"New element: {data}")
                print("Deleting old element and inserting new element")
                collection.delete_one({"split": "test", "line": num+1})
                collection.insert_one(
                    {
                        "split": "train",
                        "line": num+1,
                        "data": data
                    }
                )
            else:
                print("Duplicate element is same, skipping")

In [ ]:
with open(os.environ["TRAIN_FILE_PATH"]) as f:
    data = f.readlines()

In [ ]:
data[5467].strip()

In [ ]:
json.dumps(recursively_dump_to_json(data[5467]))